# Two transporters problem

In [1]:
import pandas as pd
from pulp import *
# pip install odfpy 

### Load the information about the products we want to transport

In [2]:
df = pd.read_excel("pdf_data.ods",
                  engine="odf", 
                  sheet_name="Sheet2"
                 )

In [3]:
df.head()

,Hardwarebenötigte,Anzahl Einheiten in Bonn,"Gewicht (mit Verpackung und Zubehör), in g",Nutzwert je Hardware-Einheit (hoch=besser)
0,"Notebook Büro 13""",205,2451,40
1,"Notebook Büro 14""",420,2978,35
2,Notebook outdoor,450,3625,80
3,Mobiltelefon Büro,60,717,30
4,Mobiltelefon Outdoor,157,988,60


In [4]:
products = list(df['Hardwarebenötigte '])

In [5]:
weights = dict(
    zip(products,df['Gewicht (mit Verpackung und Zubehör), in g'])
)
capacities = dict(zip(products, list(df['Anzahl Einheiten in Bonn'])))


In [6]:
values = dict(zip(products,df['Nutzwert je Hardware-Einheit (hoch=besser)']))

## Let's define the linear programmation problem

In [7]:
prob = LpProblem("Problem",LpMaximize)

Here we define the problem varaibles: $x_{11}, \dots ,x_{P1}, x_{12}, \dots ,x_{P2} $ that correspond to the amount transported for product $p$ with the truck $t$ 1 or 2.
the lower bound is set to zero because it is impossible to transport a negative amount of a product

$\forall p \in products, t \in \{1, 2\}, x_{pt}\geq0$

In [8]:
Transporter1 = LpVariable.dicts(
    name='Transporter1', 
    indexs=products, 
    lowBound=0,
    cat='Integer')

Transporter2 = LpVariable.dicts(
    name='Transporter2', 
    indexs=products, 
    lowBound=0,
    cat='Integer')

Here we define the objective function f to maximize
$f =   \sum_{p} v_p \times (x_{p1} + x_{p2})$

In [9]:
prob += lpSum([
    values[p]*(Transporter1[p] + Transporter2[p]) for p in products
]) #objective function

here we define the upper bound of the capacities linked to the number of products there is in storage $s$ in Bonn

$\forall p \in products, x_{p1}+x_{p2} \leq s_p$

In [10]:
for p in products:
    prob += Transporter1[p]+Transporter2[p] <= capacities[p]

we can define the constrains linked to the truck maximum weight capacity $C = 1100000 g$

$\sum_p w_p \times x_{p1} \leq C - 72400$

$\sum_p w_p \times x_{p2} \leq C - 85700$

In [11]:
#the weight of the trucks limit
prob += lpSum([weights[p]*Transporter1[p] for p in products])<=(1100*1000 - 72400)
prob += lpSum([weights[p]*Transporter2[p] for p in products])<=(1100*1000 - 85700) 

In [12]:
prob.writeLP("Problem.lp")

[Transporter1_Mobiltelefon_Büro_,
 Transporter1_Mobiltelefon_Heavy_Duty_,
 Transporter1_Mobiltelefon_Outdoor_,
 Transporter1_Notebook_Büro_13"_,
 Transporter1_Notebook_Büro_14"_,
 Transporter1_Notebook_outdoor_,
 Transporter1_Tablet_Büro_groß_,
 Transporter1_Tablet_Büro_klein_,
 Transporter1_Tablet_outdoor_groß_,
 Transporter1_Tablet_outdoor_klein_,
 Transporter2_Mobiltelefon_Büro_,
 Transporter2_Mobiltelefon_Heavy_Duty_,
 Transporter2_Mobiltelefon_Outdoor_,
 Transporter2_Notebook_Büro_13"_,
 Transporter2_Notebook_Büro_14"_,
 Transporter2_Notebook_outdoor_,
 Transporter2_Tablet_Büro_groß_,
 Transporter2_Tablet_Büro_klein_,
 Transporter2_Tablet_outdoor_groß_,
 Transporter2_Tablet_outdoor_klein_]

### Let's run the solver and see if an optimal feasable solution exists

In [13]:
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


Therefore, the optimal usage value for only one drive is

In [14]:
vs = [v for v in prob.variables() if v.varValue>0]

In [15]:
pd.DataFrame(data={
    'transporter': [v.name[11] for v in vs],
    'product': [v.name[13:-1].replace('_',' ') for v in vs],
    'quantity': [int(v.varValue) for v in vs]
})

,transporter,product,quantity
0,1,Mobiltelefon Büro,52
1,1,Mobiltelefon Heavy Duty,220
2,1,Tablet Büro klein,509
3,1,Tablet outdoor klein,4
4,2,Mobiltelefon Büro,8
5,2,Mobiltelefon Outdoor,157
6,2,Tablet Büro klein,86
7,2,Tablet outdoor groß,370


In [16]:
print("The total usage value of this drive is: {}".format(value(prob.objective)))

The total usage value of this drive is: 74660.0
